### 一、每个英雄的链接采集

In [34]:
import requests
res = requests.get('https://pvp.qq.com/web201605/js/herolist.json')
herolist_json = res.json()
for i in range(len(herolist_json)):
    herolist_json[i]['ename'] = 'https://pvp.qq.com/web201605/herodetail/{}.shtml'.format(herolist_json[i]['ename'])

herolist_json[0]

{'ename': 'https://pvp.qq.com/web201605/herodetail/105.shtml',
 'cname': '廉颇',
 'title': '正义爆轰',
 'new_type': 0,
 'hero_type': 3,
 'skin_name': '正义爆轰|地狱岩魂',
 'moss_id': 3627}

|herodetail-sort|类型|
|:-:|:-:|
|1|战士|
|2|法师|
|3|坦克|
|4|刺客|
|5|射手|
|6|辅助|

### 存入 MongoDB 数据库

In [39]:
import pymongo
client = pymongo.MongoClient(host='localhost', port=27017)
db = client['HonorOfKings']     # 指定数据库
collection = db['herolist']     # 指定集合
collection.insert_many(herolist_json)   # 插入文档

<div align=center>
<img alt="图 1" src="../images/2b51d0a655a9a13bf8bf78c88a38bb75babffa4986031ba2ec4e25d0bd0153c0.png" width=75%/>  
</div>

### 采集每个英雄的详细信息

In [51]:
# 读取数据库
from unittest import result


client = pymongo.MongoClient(host='localhost', port=27017)
db = client['HonorOfKings']
collection = db['herolist']

results = collection.find({})

In [58]:
for result in results:
    # print(result)
    print(result['ename'])
    break

https://pvp.qq.com/web201605/herodetail/106.shtml


In [55]:
import requests
url = 'https://pvp.qq.com/web201605/herodetail/105.shtml'
res = requests.get(url)
res.encoding = 'gbk'
from lxml import etree
html = etree.HTML(res.text)

#### 英雄基本信息【完成】

In [8]:
# 英雄基本信息
def get_cover_info(html):
    # 英雄基本信息
    cover      = html.xpath('//div[@class="cover"]')[0]
    
    cover_name = cover.xpath('./h2/text()')[0]      # 英雄名
    herodetail_sort = cover.xpath('./span/i/@class')[0]     # 英雄类型
    cover_list = []
    for li in cover.xpath('.//ul/li'):
        cover_list_name = li.xpath('./em/text()')[0]
        cover_list_bar  = li.xpath('./span/i/@style')[0]
        cover_list.append({'cover_list_name':cover_list_name, 'cover_list_bar':cover_list_bar})
    cover_info = {
        'cover_name':cover_name, 
        'herodetail_sort':herodetail_sort[-1:], 
        'cover_list':cover_list
        }
    return cover_info

get_cover_info(html)

{'cover_name': '廉颇',
 'herodetail_sort': '3',
 'cover_list': [{'cover_list_name': '生存能力', 'cover_list_bar': 'width:100%'},
  {'cover_list_name': '攻击伤害', 'cover_list_bar': 'width:30%'},
  {'cover_list_name': '技能效果', 'cover_list_bar': 'width:40%'},
  {'cover_list_name': '上手难度', 'cover_list_bar': 'width:30%'}]}

#### 技能介绍【完成】

In [3]:
# 技能介绍
def get_skill_info_details(html):
    # 技能介绍
    skill_info = html.xpath('//div[contains(@class, "skill-show")]')[0]
    
    skill_info_details = []
    for div in skill_info.xpath('./div'):
        if len(div.xpath('./p[1]/b/text()')) == 0:
            pass
        else:
            skill_name = div.xpath('./p[1]/b/text()')[0]            # 技能名
            skill_time = div.xpath('./p[1]/span[1]/text()')[0][4:].split('/')  # 技能冷却值
            skill_need = div.xpath('./p[1]/span[2]/text()')[0][3:]  # 技能消耗
            skill_desc = div.xpath('./p[2]/text()')                 # 技能描述
            skill_info_details.append({
                'skill_name' : skill_name,
                'skill_time' : skill_time,
                'skill_need' : skill_need,
                'skill_desc' : skill_desc,
            })

    return skill_info_details

get_skill_info_details(html)

[{'skill_name': '勇士之魂',
  'skill_time': ['0'],
  'skill_need': '0',
  'skill_desc': ['被动：廉颇释放技能过程中会处于霸体状态。进入战斗后廉颇的战意会逐渐提升，战意越高减伤越多，最多获得20%所受到的伤害。当战意充盈时，廉颇提升自身30%攻击速度。脱战后战意逐渐降低。']},
 {'skill_name': '爆裂冲撞',
  'skill_time': ['9'],
  'skill_need': '0',
  'skill_desc': ['廉颇向指定方向发起冲锋，对敌人造成150/170/190/210/230/250（+65%物理攻击）点物理伤害并将其击飞0.5秒。 被动：第三次普攻为直拳重击，对敌人造成额外200/260/320/380/440/500（+15%物理加成）点物理伤害。']},
 {'skill_name': '熔岩重击',
  'skill_time': ['11', '10.4', '9.8', '9.2', '8.6', '8'],
  'skill_need': '0',
  'skill_desc': ['廉颇迸发狂暴力量并通过臂甲传入地面，获得350点护盾，1秒后地面会崩裂，对区域内的敌人造成100/125/150/175/200/225（+100%物理加成）（+4%最大生命加成）物理伤害，距离中心越近伤害越高，最高提升100%。技能范围内有敌人时会重置爆裂冲撞的冷却时间，并减少范围内敌人30%移动速度，非英雄单位会被拉向范围中心。技能释放期间可以使用其他技能。']},
 {'skill_name': '天崩地裂',
  'skill_time': ['54', '45', '36'],
  'skill_need': '0',
  'skill_desc': ['廉颇跳向指定区域连续锤击地面三次，第一次：造成60/90/120（+60%物理加成）（+3%最大生命）点物理伤害和30%减速。第二次：造成90（+90%物理加成）（+4%最大生命）点物理伤害和50%减速。第三次：造成120（+150%物理加成）（+6%最大生命）点物理伤害和1秒击飞。']}]

#### 铭文搭配建议【❌】

In [56]:
# 铭文搭配建议【暂时】
sugg_info  = html.xpath('//ul[@class="sugg-u1"]')[0]
sugg_tips  = html.xpath('//p[@class="sugg_tips"]')
sugg_tips

[]

<img alt="图 1" src="../images/a3e77f5ee5dd0ecdc7d6cc5383f41337531a48368d799d2d75dbd43b993c17a5.png" />  

<img alt="图 2" src="../images/080b6774185623978cdbcdb74d361241f77fa8b118abae82c454097d9c8bbd42.png" />  


#### 技能加点建议【完成】

In [72]:
# 技能加点建议
def get_skill_upgrade_sugg(html):
    sugg_info2 = html.xpath('//div[contains(@class, "sugg-info2")]/p[contains(@class, "sugg-name")]')

    major_skill_sugg = sugg_info2[0].xpath('./span/text()')[0]
    minor_skill_sugg = sugg_info2[1].xpath('./span/text()')[0]
    hero_skill_sugg  = sugg_info2[2].xpath('./span/text()')[0].split('/')

    skill_upgrade_sugg = {
        'major_skill_sugg' : major_skill_sugg,
        'minor_skill_sugg' : minor_skill_sugg,
        'hero_skill_sugg' : hero_skill_sugg,
    }

    return skill_upgrade_sugg

get_skill_upgrade_sugg(html)

{'major_skill_sugg': '火焰三尖枪',
 'minor_skill_sugg': '混天绫.束',
 'hero_skill_sugg': ['闪现', '斩杀']}

#### 英雄关系【完成】

In [130]:
# 英雄关系
def get_hero_relationship(html):
    hero_info_box = html.xpath('//div[@class="hero-info-box"]/div/div')

    def hero_info(xpath_):
        ids = [i.split('.')[0] for i in xpath_.xpath('./div[2]/ul/li/a/@href')]
        tips = xpath_.xpath('./div[3]/p/text()')
        
        relationship_dict = []
        for item in list(zip(ids, tips)):
            relationship_dict.append({
                'id':item[0],
                'tip':item[1]
            })
        return relationship_dict
    
    hero_relationship = {}
    for i in range(len(hero_info_box)):
        if i == 0:
            hero_relationship['最佳搭档'] = hero_info(xpath_=hero_info_box[i])
        elif i == 1:
            hero_relationship['压制英雄'] = hero_info(xpath_=hero_info_box[i])
        else:
            hero_relationship['被压制英雄'] = hero_info(xpath_=hero_info_box[i])
      
    return hero_relationship

get_hero_relationship(html)

{'最佳搭档': [{'id': '116',
   'tip': '阿轲：廉颇拥有比较强的坦克能力，在团战和对线时都可以主动利用1技能开团后2技能消耗找敌方英雄换血，等到敌方英雄交出关键技能后，可以利用自己的13技能控制配合阿轲完成击杀和收割。'},
  {'id': '132',
   'tip': '马可波罗：马可波罗拥有比较强的远程poke能力，2技能的突进也可以快速跟上廉颇的1技能突进，对线时廉颇可以利用1技能和3技能控制限制敌方刺客对己方马可波罗的刺杀，也可以主动利用1技能开团，3技能配合闪现打出双段控制，为马可波罗创造良好的输出环境。'}],
 '压制英雄': [{'id': '156',
   'tip': '张良：张良作为一个比较依赖控制效果的法师，1技能的控制和大招的控制，对拥有免伤的廉颇来说效果甚微，而且廉颇的1技能和3技能都可以很好的打断张良的大招技能对己方进行保护，为队友提供良好的生存环境。'},
  {'id': '152',
   'tip': '王昭君：作为一个比较依赖2技能控制效果的英雄，面对廉颇时会很尴尬，因为廉颇本身拥有比较强的坦克能力，即使被王昭君大到也很难一套秒杀，王昭君开大后廉颇1技能和3技能都可以轻松打断，进攻时利用控制技能也配合队友黏住王昭君轻松将其击杀。'}],
 '被压制英雄': [{'id': '169', 'tip': '后羿普攻有减速效果，很容易风筝廉颇'},
  {'id': '141', 'tip': '廉颇不能很容易控制到灵活的貂蝉'}]}

#### 出装建议【❌】

<img alt="图 1" src="../images/f7558dc6ac5f39a66851bbba2b9b953c100b15e8aaaffd8417f64104775d5172.png" />  


In [146]:
# 出装建议

equips_bd = html.xpath('//div[@class="equip-bd"]/div')

equip_list = []
for equip_bd in equips_bd:
    # equip_info
    for li in equip_bd.xpath('./ul/li'):
        # print(li)
        break

    # tips
    tips = equip_bd.xpath('./p/text()')[0][5:]
    '''
    # 合并成字典加入列表
    equip_list.append({
        'equip_info':,
        'equip_tips':tips
    })
    '''
    break

提高生存能力，在团战中控制敌人为队友提供更多的操作空间。
抓准时间进入战场利用位移技能接近敌方英雄并释放范围技能可以打出更多的伤害。


In [149]:
a = equips_bd[0]
a.xpath('')

['\n                            ',
 '\n                                ',
 '\n                            ',
 '\n                            ',
 'Tips：提高生存能力，在团战中控制敌人为队友提供更多的操作空间。',
 '\n                        ']